In [ ]:
# !pip install -U tqdm

In [ ]:
# !pip install bentoml --upgrade

In [1]:
import bentoml
from transformers import pipeline
import torch
from bentoml.adapters import JsonInput
from bentoml.artifact import PickleArtifact

[2020-11-15 09:40:26,241] WARNING - Importing from "bentoml.artifact.*" has been deprecated. Instead, use`bentoml.frameworks.*` and `bentoml.service.*`. e.g.:, `from bentoml.frameworks.sklearn import SklearnModelArtifact`, `from bentoml.service.artifacts import BentoServiceArtifact`, `from bentoml.service.artifacts.common import PickleArtifact`


In [5]:
%%writefile transformer_service.py
import bentoml
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from bentoml.adapters import JsonInput
from bentoml.service.artifacts.common import PickleArtifact
import torch
import math
import warnings
import numpy as np

@bentoml.env(pip_packages=["transformers==3.1.0", "torch==1.6.0"])
@bentoml.artifacts([PickleArtifact('bartModel')])
class TransformerService(bentoml.BentoService):
    @bentoml.api(input=JsonInput(), batch=False)
    def predict(self, payload):
        # Initialize labels
        risk_labels = ["Data Collection", 
                            "Data Privacy", 
                            "Data Processes", 
                            "Data Modelling", 
                            "Data Encryption",
                            "Data Security", 
                            "Not Related to Data"]
                
        # Natural Language Inference Premise inputs
        nli_template_risk_label = "The risk factor of this text is on {}."  
        
        # Get text paypload
        response = payload["response"]
        
        # JSON output
        risk_dict = self.artifacts.bartModel(response, risk_labels, hypothesis_template=nli_template_risk_label)
        
        return risk_dict

Overwriting transformer_service.py


In [6]:
from transformer_service import TransformerService
ts = TransformerService()

In [7]:
model_name = "facebook/bart-large-mnli"
classifier = pipeline("zero-shot-classification",  model=model_name)


Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
ts.pack("bartModel", classifier)

In [9]:
saved_path = ts.save()

[2020-11-15 09:41:37,426] INFO - BentoService bundle 'TransformerService:20201115094127_C00397' saved to: /Users/xavierpuspus/bentoml/repository/TransformerService/20201115094127_C00397


In [10]:
saved_path

'/Users/xavierpuspus/bentoml/repository/TransformerService/20201115094127_C00397'

In [ ]:
### Try it using bentoml serve {saved_path}